[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/seap-udea/tspice/blob/main/examples/love_numbers_indepth.ipynb)

# Love numbers using `tSPICE`: In depth

This notebook demonstrates how to calculate the tidal potential generated by various celestial bodies at a specific location on Earth using `tSPICE`.

In [1]:
try:
    from google.colab import drive
    %pip install -Uq git+https://github.com/DeivyMercado/TSPICE
except ImportError:
    print("Not running in Colab, skipping installation")
    %load_ext autoreload
    %autoreload 2

# Uncomment to install from GitHub (development version)
# !pip install git+https://github.com/seap-udea/tspice.git

Not running in Colab, skipping installation


In [3]:
import numpy as np
import tspice as tsp
tsp.initialize()

TSPICE initialized successfully. Kernels loaded from: /Users/jzuluaga/dev/tspice/examples/tspice_data/meta_kernel


In [4]:
from tspice.planet import Earth
earth_model = Earth()
planet_profile = earth_model.planet_profile

In [5]:
#Create Earth interior model
earth_interior = tsp.BodyResponse('Earth')

#Define layers for our planetary model
earth_interior.scale_constants(verbose=True)
layers_list = [dict(name='Outer Core', type='fluid', r0=1221500.0, rf=3480000.0),
               dict(name='Inner Core', type='solid', r0=0, rf=1221500.0),
               dict(name='Mantle + crust', type='solid', r0=3480000.0, rf = earth_interior.L)]


Characteristic scales for adimensionalization:
Length scale L = 6.37e+06 m
Mass scale M = 5.97e+24 kg
Density scale RHO = 2.31e+04 kg/m^3
Pressure/Elasticity scale P = 1.44e+12 Pa
Velocity scale V = 7.91e+03 m/s
Time scale T = 8.05e+02 s
Angular frequency scale OMEGA = 1.24e-03 rad/s
Gravity scale Gad = 9.82e+00 m/s^2


## Comparison with Amorin & Gudkova (2024):

In [6]:
#Set parameters for the integration	
n = 2
f_day = 1.9323  #M2 tide in cycles/day
r0_ini = 6e3
earth_interior.set_integration_parameters_ad(n=n, f_days=f_day, layers_list=layers_list, planet_profile=planet_profile, r0_ini=r0_ini)

#Integration
earth_interior.integrate_internal_solutions_ad(verbose=False)

#Get Love numbers
h_2_tsp, l_2_tsp, k_2_tsp = earth_interior.h_n, earth_interior.l_n, earth_interior.k_n
print(f"Computed Love numbers for n={n} at f={f_day} cpd:")
print(f"h_{n} = {h_2_tsp}")
print(f"l_{n} = {l_2_tsp}")
print(f"k_{n} = {k_2_tsp}")

Computed Love numbers for n=2 at f=1.9323 cpd:
h_2 = 0.6093104505940026
l_2 = 0.08563962156161709
k_2 = 0.29904089799774747


In [7]:
#Set parameters for the integration	
n = 3
f_day = 1.9323  #M2 tide in cycles/day
r0_ini = 6e3
earth_interior.set_integration_parameters_ad(n=n, f_days=f_day, layers_list=layers_list, planet_profile=planet_profile, r0_ini=r0_ini)

#Integration
earth_interior.integrate_internal_solutions_ad(verbose=False)

#Get Love numbers
h_3_tsp, l_3_tsp, k_3_tsp = earth_interior.h_n, earth_interior.l_n, earth_interior.k_n
print(f"Computed Love numbers for n={n} at f={f_day} cpd:")
print(f"h_{n} = {h_3_tsp}")
print(f"l_{n} = {l_3_tsp}")
print(f"k_{n} = {k_3_tsp}")

Computed Love numbers for n=3 at f=1.9323 cpd:
h_3 = 0.30487631311183616
l_3 = 0.030899353905976534
k_3 = 0.09285112727873845


In [8]:
#Amorim & Gudkova (2025) results
h_2_ag = 0.60496
l_2_ag = 0.08399
k_2_ag = 0.29872
k_3_ag = 0.09203

In [9]:
#Difference between TSPICE and Amorim & Gudkova (2025)
print("\nDifferences with Amorim & Gudkova (2025) results:")
print(f"Delta h_2 = {h_2_tsp - h_2_ag:.4f}")
print(f"Delta l_2 = {l_2_tsp - l_2_ag:.4f}")
print(f"Delta k_2 = {k_2_tsp - k_2_ag:.4f}")
print(f"Delta k_3 = {k_3_tsp - k_3_ag:.4f}")


#Percentage differences
print("\nPercentage differences with Amorim & Gudkova (2025) results:")
print(f"Percentage Delta h_2 = {(h_2_tsp - h_2_ag)/h_2_ag * 100:.3f} %")
print(f"Percentage Delta l_2 = {(l_2_tsp - l_2_ag)/l_2_ag * 100:.3f} %")
print(f"Percentage Delta k_2 = {(k_2_tsp - k_2_ag)/k_2_ag * 100:.3f} %")
print(f"Percentage Delta k_3 = {(k_3_tsp - k_3_ag)/k_3_ag * 100:.3f} %")


Differences with Amorim & Gudkova (2025) results:
Delta h_2 = 0.0044
Delta l_2 = 0.0016
Delta k_2 = 0.0003
Delta k_3 = 0.0008

Percentage differences with Amorim & Gudkova (2025) results:
Percentage Delta h_2 = 0.719 %
Percentage Delta l_2 = 1.964 %
Percentage Delta k_2 = 0.107 %
Percentage Delta k_3 = 0.892 %


## Comparison with Xu & Sun (2003)

In [10]:
love_numbers_xu = np.array([[0.603306, 0.298256, 0.083949],
          [0.287109, 0.091773, 0.014849],
          [0.174801, 0.041418, 0.010151],
          [0.605630, 0.299365, 0.084042],
          [0.288168, 0.092162, 0.014698],
          [0.174977, 0.041437, 0.010147],
		[0.288848, 0.092362, 0.014644],
        [0.175158, 0.041479, 0.010134],
        [0.176197, 0.041700, 0.010060]])
love_numbers_xu

array([[0.603306, 0.298256, 0.083949],
       [0.287109, 0.091773, 0.014849],
       [0.174801, 0.041418, 0.010151],
       [0.60563 , 0.299365, 0.084042],
       [0.288168, 0.092162, 0.014698],
       [0.174977, 0.041437, 0.010147],
       [0.288848, 0.092362, 0.014644],
       [0.175158, 0.041479, 0.010134],
       [0.176197, 0.0417  , 0.01006 ]])

In [11]:
#To store the results and comparison
love_numbers_tsp = np.zeros_like(love_numbers_xu)
deltas = np.zeros_like(love_numbers_xu)
deltas_percent = np.zeros_like(love_numbers_xu)
love_numbers_tsp

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [12]:
#Frequencies to compare
fO1 = 0.92953571
fM2 = 1.9323
fM3 = 2.8984
fM4 = 3.8645
fs = [fO1, fO1, fO1, fM2, fM2, fM2, fM3, fM3, fM4]

#List of degrees
ns = [2,3,4,2,3,4,3,4,4]

#Cycles per day
for i,(f,n) in enumerate(zip(fs,ns)):
	
	#Set parameters for the integration
	earth_interior.set_integration_parameters_ad(n=n, f_days=f, layers_list=layers_list, planet_profile=planet_profile, r0_ini=r0_ini)

	#Integration
	earth_interior.integrate_internal_solutions_ad(verbose=False)

	#Get Love numbers
	h_n_tsp, l_n_tsp, k_n_tsp = earth_interior.h_n, earth_interior.l_n, earth_interior.k_n
	love_numbers_tsp[i,:] = [h_n_tsp, k_n_tsp, l_n_tsp]
	deltas[i,:] = love_numbers_tsp[i,:] - love_numbers_xu[i,:]
	deltas_percent[i,:] = deltas[i,:] / love_numbers_xu[i,:] * 100


	#print(f"Computed Love numbers for n={n} at f={f} cpd:")
	#print(f"h_{n} = {h_n_tsp}")
	#print(f"l_{n} = {l_n_tsp}")
	#print(f"k_{n} = {k_n_tsp}")

	#

In [13]:
for i,(f,n) in enumerate(zip(fs,ns)):
    print(f"Frequency: {f} cpd, Degree: {n}")

Frequency: 0.92953571 cpd, Degree: 2
Frequency: 0.92953571 cpd, Degree: 3
Frequency: 0.92953571 cpd, Degree: 4
Frequency: 1.9323 cpd, Degree: 2
Frequency: 1.9323 cpd, Degree: 3
Frequency: 1.9323 cpd, Degree: 4
Frequency: 2.8984 cpd, Degree: 3
Frequency: 2.8984 cpd, Degree: 4
Frequency: 3.8645 cpd, Degree: 4


---
**Copyright**: 2026, Deivy Mercado, Jorge I. Zuluaga, Gloria Moncayo